In [1]:
from sklearn.model_selection import train_test_split

In [3]:
import pandas as pd

dataset = pd.read_csv('../data/df_texts_ocean.csv')
dataset.head()

,user_id,post_text,Экстраверсия,Доброжелательность,Сознательность,Нейротизм,Открытость опыту,Экстраверсия_3,Доброжелательность_3,Сознательность_3,Нейротизм_3,Открытость опыту_3,Экстраверсия_5,Доброжелательность_5,Сознательность_5,Нейротизм_5,Открытость опыту_5,post_text_preprocessed
0,133,Открытое образование - Главная страница,4.88,4.11,4.67,1.13,4.9,2,2,2,0,2,4,3,4,0,4,открытое образование PUNK главная страница
1,133,Первый день форума «Бережливое управление 2.0»,4.88,4.11,4.67,1.13,4.9,2,2,2,0,2,4,3,4,0,4,первый день форума PUNK бережливое управление ...
2,133,Есть контакт | Центры «Мои Документы» Липецкой...,4.88,4.11,4.67,1.13,4.9,2,2,2,0,2,4,3,4,0,4,есть контакт PUNK центры PUNK мои документы PU...
3,133,До аэропорта «Липецк» будет ездить бесплатный ...,4.88,4.11,4.67,1.13,4.9,2,2,2,0,2,4,3,4,0,4,до аэропорта PUNK липецк PUNK будет ездить бес...
4,133,https://vk.com/wall-171900971_843,4.88,4.11,4.67,1.13,4.9,2,2,2,0,2,4,3,4,0,4,URL


In [4]:
from sklearn.utils import shuffle

def split_with_owersampling(dataset, prediction_column):

    train_data, valid_data = train_test_split(dataset, test_size=0.3, stratify=dataset[prediction_column], random_state=42)
    valid_data, test_data = train_test_split(valid_data, test_size=0.5, stratify=valid_data[prediction_column], random_state=42)

    if 'Экстраверсия' in prediction_column:
        train_data = pd.concat([train_data,
                        train_data[train_data[prediction_column]==4],
                        train_data[train_data[prediction_column]==4],
                        train_data[train_data[prediction_column]==0],
                        train_data[train_data[prediction_column]==0],
                        train_data[train_data[prediction_column]==0],
                        train_data[train_data[prediction_column]==0]])
        
    if 'Доброжелательность' in prediction_column:
        # отнимаем 1 потому что в доброжелательности нет нулевого класса и лосс ругается
        train_data[prediction_column] = train_data[prediction_column].apply(lambda x: x-1)
        valid_data[prediction_column] = valid_data[prediction_column].apply(lambda x: x-1)
        test_data[prediction_column] = test_data[prediction_column].apply(lambda x: x-1)
        train_data = pd.concat([train_data,
                        train_data[train_data[prediction_column]==0],
                        train_data[train_data[prediction_column]==0],
                        train_data[train_data[prediction_column]==0],
                        train_data[train_data[prediction_column]==0],
                        train_data[train_data[prediction_column]==3],
                        train_data[train_data[prediction_column]==3],
                        train_data[train_data[prediction_column]==3],
                        train_data[train_data[prediction_column]==3],])
        
    if 'Сознательность' in prediction_column:
        train_data = pd.concat([train_data,
                        train_data[train_data[prediction_column]==1],
                        train_data[train_data[prediction_column]==4],
                        train_data[train_data[prediction_column]==0],
                        train_data[train_data[prediction_column]==0],
                        train_data[train_data[prediction_column]==0],
                        train_data[train_data[prediction_column]==0],
                        train_data[train_data[prediction_column]==0],
                        train_data[train_data[prediction_column]==0]])
        
    if 'Нейротизм' in prediction_column:
        train_data = pd.concat([train_data,
                        train_data[train_data[prediction_column]==0]])
    
    if 'Открытость' in prediction_column:
        train_data = pd.concat([train_data,
                        train_data[train_data[prediction_column]==0],
                        train_data[train_data[prediction_column]==1],
                        train_data[train_data[prediction_column]==0].head(5000),
                        train_data[train_data[prediction_column]==1].head(5000)])


    train_data = shuffle(train_data)
    train_data.reset_index(inplace=True, drop=True)
    valid_data.reset_index(inplace=True, drop=True)
    test_data.reset_index(inplace=True, drop=True)

    return train_data, valid_data, test_data

In [6]:
prediction_column = 'Нейротизм_5'
train_data, valid_data, test_data = split_with_owersampling(dataset, prediction_column)
train_data[prediction_column].value_counts(ascending=True)

Нейротизм_5
0    13978
4    17230
1    25426
3    56078
2    69087
Name: count, dtype: int64

In [7]:
dataset[prediction_column].value_counts(ascending=True)

Нейротизм_5
0     9984
4    24614
1    36323
3    80112
2    98696
Name: count, dtype: int64

In [ ]:
#!/bin/bash
#SBATCH --job-name=my_job           # Job name
#SBATCH --output=output_%j.txt      # Output file name (%j expands to job ID)
#SBATCH --error=error_%j.txt        # Error file name (%j expands to job ID)
#SBATCH --ntasks=1                  # Number of tasks (one task per script instance)
#SBATCH --cpus-per-task=1           # Number of CPU cores per task
#SBATCH --mem=1G                    # Memory per node
#SBATCH --time=00:10:00             # Walltime limit (hh:mm:ss)

# Load any necessary modules
# module load python

# Activate your virtual environment if required
# source /path/to/your/virtualenv/bin/activate

# Change to the directory where your Python script is located
cd /path/to/your/python/script

# Define arrays for each argument
learning_rates=(0.1 0.01 0.001)
dropout_rates=(0.2 0.5 0.8)
use_augmentation=(true false)

# Iterate over the arguments and run the Python script for each combination
for lr in "${learning_rates[@]}"
do
    for dropout in "${dropout_rates[@]}"
    do
        for augment in "${use_augmentation[@]}"
        do
            echo "Running script with learning rate: $lr, dropout: $dropout, augmentation: $augment"
            python your_script.py --lr $lr --dropout $dropout --augmentation $augment
        done
    done
done
